### MCLP

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely import wkt
import numpy as np

In [ ]:
df=pd.read_csv('격자.csv')
df=gpd.GeoDataFrame(df)
df.set_crs(epsg=3857)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# 스케일링할 열 목록
columns_to_scale = ['교통혼잡도', '인구수', '건물수', '공시지가' , '불법주정차','주차구획수']

# 스케일링 진행
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [ ]:
df['value']=df['교통혼잡도']*0.35+df['인구수']*0.137+df['건물수']*0.191+df['공시지가']*0.136+df['불법주정차'].fillna(0)*0.185

In [ ]:
facility_df=df[df['주차구획수'].isna()].reset_index(drop=True)
top_criterion = np.quantile(facility_df['value'],0.75)
print(np.quantile(facility_df['value'],0.75))
demand_df = facility_df.query('value > @top_criterion').reset_index(drop=True)

0.3135579009607047


In [ ]:
f_centroid = facility_df['geometry'].centroid   # 후보입지 격자의 중심좌표
d_centroid = demand_df['geometry'].centroid     # 수요입지 격자의 중심좌표

In [ ]:
from shapely.geometry import Point

# 중심좌표 및 중심좌표 주위 8개의 점을 생성하기 위한 연산 리스트
add_list = [[0,0]]

# 수요격자의 중심좌표 포함 9개 좌표 생성 (+-33m)
def add_demand(x,y,weight):  # weight : 해당 격자의 범죄위험지수
    temp_list=[]
    
    for add in add_list:
        new_x, new_y = x+add[0] ,y+add[1]
        temp_list.append((new_x, new_y,weight))
    return temp_list

# 후보격자의 중심좌표 포함 9개 좌표 생성 (+-33m)
def add_facility(x,y):
    temp_list=[]
    
    for add in add_list:
        new_x, new_y = x+add[0] ,y+add[1]
        temp_list.append((new_x, new_y))
    return temp_list


In [ ]:
## 수요격자 내 일정 간격의 좌표 생성
demand_points = []

for index in range(len(demand_df)): 
    weight = demand_df['value'][index]
    y,x = d_centroid[index].coords.xy      # 수요격자의 중심좌표
    
    # 중심좌표 포함 9개 좌표 생성 (+-33m)
    demand_points += add_demand(x[0],y[0],weight)            
    

In [ ]:
## 후보격자 내 일정 간격의 좌표 생성
facility_points = []

for index in range(len(facility_df)): 
    y,x = f_centroid[index].coords.xy      # 후보격자의 중심좌표
    # 중심좌표 포함 9개 좌표 생성 (+-33m)
    facility_points += add_facility(x[0],y[0])    
    

In [ ]:
print(len(demand_df)*9 , len(demand_points))
print(len(facility_df)*9 , len(facility_points))

4914 546
19638 2182


In [ ]:
grid_points = []   # 격자의 외곽 좌표 리스트

# 격자의 외곽 좌표를 후보지점에 추가
for index in range(len(facility_df)):
    
    polygon = facility_df['geometry'][index]
    polygon_points = list(polygon.exterior.coords)
    polygon_points2 = [(lat, lon) for lon, lat in polygon_points]
    grid_points += polygon_points2

unique_grid_points = list(set(grid_points))
print(len(unique_grid_points))

2521


In [ ]:
# 최종 후보좌표  : 후보격자 내부 좌표 9곳 + 외곽 좌표 4곳 (중복 제외)
final_facility_points = facility_points + unique_grid_points

In [ ]:
exclusion_df=pd.read_csv('최종 데이터/주차장/주차장_변환_.csv')

In [ ]:
exclusion_df.columns=['d', '위도', '경도', 'ㅇ', 'ㅇ', 'ㅇ']

In [ ]:
exclusion_points = []

for index in range(len(exclusion_df)):
    x,y = exclusion_df['위도'][index] ,exclusion_df['경도'][index]
    exclusion_points.append((x,y))

# MCLP 모델링 

In [ ]:
df=df.to_crs(epsg=4326)

In [ ]:
from shapely.geometry import Polygon

# 경도와 위도 위치 바꾸기
def swap_lat_lon(geom):
    if geom.geom_type == 'Polygon':
        return Polygon([(y, x) for x, y in geom.exterior.coords])
    elif geom.geom_type == 'MultiPolygon':
        return MultiPolygon([Polygon([(y, x) for x, y in part.exterior.coords]) for part in geom])
    else:
        raise ValueError("지원되지 않는 지오메트리 타입입니다.")

# 적용
df['geometry'] = df['geometry'].apply(swap_lat_lon)

In [ ]:
import pulp
from geopy.distance import geodesic

final_cctv_index = []
covered_info = []

cover_radius = 0.1  # km 단위, 50m
exclusion_radius = 0.1 # km 단위, 50m
max_facilities = 5  # 최대 설치 가능한 CCTV 수


# 모델 설정
model = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 변수
x = pulp.LpVariable.dicts("x", range(len(facility_points)), cat='Binary')  # 각 설비의 설치 여부
y = pulp.LpVariable.dicts("y", ((i, j) for i in range(len(demand_points)) for j in range(len(facility_points))), cat='Binary')  # 수요 지점 i가 설비 j에 의해 서비스되는지 여부

# 목적 함수
model += pulp.lpSum(demand_points[i][2] * y[i, j] for i, j in y)  # 가중치를 반영한 커버 최대화

# 제약 조건
# 설치할 수 있는 CCTV의 총 수를 제한
model += pulp.lpSum(x[j] for j in range(len(facility_points))) == max_facilities

# 특정 지역 반경 내의 후보 지점 제외
for i, facility in enumerate(facility_points):
    for j, exclusion_center in enumerate(exclusion_points):
        if geodesic(exclusion_center, facility).km <= exclusion_radius:
            model += x[i] == 0  # 제외 범위 내의 설비는 설치 불가

for i, demand in enumerate(demand_points):
    for j, facility in enumerate(facility_points):
        # 수요 지점과 설비 사이의 거리가 서비스 범위 내인지 확인
        if geodesic(demand[:2], facility).km <= cover_radius:
            model += y[i, j] <= x[j]  # 수요 지점 i가 설비 j에 의해 서비스되려면 설비 j가 설치되어야 함
        else:
            model += y[i, j] == 0  # 거리가 너무 멀면 서비스 불가능

# 각 수요 지점은 하나의 설비에 의해서만 서비스될 수 있음
for i in range(len(demand_points)):
    model += pulp.lpSum(y[i, j] for j in range(len(facility_points))) <= 1

# 문제 해결
model.solve()

# 결과 출력
print("Selected Facilities and their Coordinates:")
for j in range(len(facility_points)):
    if pulp.value(x[j]) == 1:
        final_cctv_index.append(j)
        print(f"Facility {j} is selected at coordinates {facility_points[j]}.")

print("Coverage and Weights:")
for i, j in y:
    if pulp.value(y[i, j]) == 1:
        covered_info.append((i,j))
        print(f"Demand point {i} with weight {demand_points[i][2]} is covered by facility {j}.")

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

index = list(range(len(final_cctv_index)))
index_df = pd.DataFrame(index,columns=['new_cctv_id'])

result =[]

for index in final_cctv_index:
    y, x = facility_points[index]
    result.append(Point(x,y))

result_df = gpd.GeoDataFrame(index_df, geometry=result)
result_df

,new_cctv_id,geometry
0,0,POINT (127.08855 37.53826)
1,1,POINT (127.06699 37.54719)
2,2,POINT (127.07154 37.54360)
3,3,POINT (127.07378 37.54632)
4,4,POINT (127.06811 37.54810)
5,5,POINT (127.07949 37.53912)
6,6,POINT (127.07605 37.54632)
7,7,POINT (127.07496 37.53911)
8,8,POINT (127.06927 37.54359)
9,9,POINT (127.07606 37.54452)
